# Notebook activity 5.10 Assessing predictions

In Notebook activity 5.9, stepwise regression was used to select a parsimonious model for the the number of medals a nation wins at a summer Olympics. The model selected was:

`medals  ~ lagMedals + population + logPopulation + logGdp + host + nextHost`.

Although the diagnostic plots for this model raised several issues, in this notebook activity we will use this model to predict the number of medals each nation wins at Rio 2016 and compare these predictions with what actually happened.

## (a) Getting started

### (i)

Start by running the following code.

In [ ]:
# Load the M348 package
library(M348)
# Load the olympic data frame
data(olympic)
# Check the data frame has loaded OK
head(olympic)

### (ii)

Run the code below to 
* create two new variables: `logPopulation` and `logGdp` and add them to the `olympic` data frame
* fit the model `medals  ~ lagMedals + population + logPopulation + logGdp + host + nextHost`, storing the results in an object called `medalsMLRFit`.

In [2]:
olympic$logPopulation <- log(olympic$population)
olympic$logGdp <-  log(olympic$gdp)
medalsMLRFit <- lm(medals  ~ lagMedals + population + logPopulation + 
                   logGdp + host + nextHost, data = olympic)

## (b) Obtaining predictions

Recall from Notebook activity 1.19 in Unit 1 and Notebook activity 2.3 in Unit 2, that predictions from a multiple regression model can be obtained using the command `predict()`. 

Recall that when using `predict()` in those notebooks, you included three arguments:

* the name of the object storing the results of the fitted regression model (which here is `medalsMLRFit`),
* the name of a data frame storing the new values of the explanatory variables for the observations you want to predict, and
* the type of interval that should be calculated, which in this case is `'prediction'`.

In this notebook activity, we want to generate predictions about the numbers of medals won by nations at Rio 2016. Specifying the first and third arguments for `predict()` are straightforward. 

Creating the data frame for the second argument of `predict()` is, however, a little bit more tricky. We only need values of the explanatory variables relating to Rio 2016, not *all* of the Olympics in our dataset. 

Recall from Notebook activity 5.5 that it is possible to select rows of a data frame using logical statements. To do this, we need a means of distinguishing which rows correspond to Rio 2016 and which do not.  The variable `year` gives us a means of doing this. This variable gives the year of the Olympics each row relates to. So, for rows relating to Rio 2016, the value is 2016, and for all other rows the value is something other than 2016. 

### (i) 

Use the empty code cell below to select all of the rows in `olympic` for which `olympic$year == 2016`. Save the resulting data frame as a new data frame called `olympic2016`.

#### Solution <a class="tocSkip">

The required code for creating the new data frame is given below. 

Note that, because we only want our new data frame to include those observations for which `year` takes the value 2016, we need to select data based on the condition `year == 2016`. What's more, we want to select entire rows of data for each of the Rio 2016 observations, and so we use `[year == 2016,]`: this specifies a condition on the rows that are selected for the new data frame, but doesn't put any restrictions on the columns (since there is nothing after the comma in the command) so that data for all of the variables in the Rio 2016 rows are included. 

In [3]:
olympic2016 <- olympic[olympic$year == 2016,]

###  <a class="tocSkip">

So having got a data frame, with the right variables in it and the data only relating to Rio 2016, we are ready to use the command `predict()`.

### (iii)

Using the empty code cell below obtain predictions, along with 95% prediction intervals, for the number of medals at Rio 2016 based on the following model:

`medals  ~ lagMedals + population + logPopulation + logGdp + host + nextHost`.

(Look back at Notebook activity 2.3 if you need a reminder of how to do this.)

Save the results in an object called `predictions2016` and display the first few lines of `predictions2016`. 

#### Solution <a class="tocSkip">

The required code and resulting output are given below.

In [4]:
predictions2016 <- predict(medalsMLRFit, newdata = olympic2016, 
                           interval = "predict")
head(predictions2016)

fit         lwr       upr     
1297  1.13880466 -4.933995 7.211605
1299  0.19886398 -5.867864 6.265592
1300  1.57896567 -4.492097 7.650029
1301  0.09897847 -5.981687 6.179644
1302  0.52903941 -5.540697 6.598776
1303 -0.10644780 -6.184041 5.971145

### (iv)

Do the predictions you obtained in part (iii) look reasonable?

#### Solution <a class="tocSkip">

Looking at the first six entries it is noticable that the predictions could be improved on as all of the lower bounds are negative when the worst that a nation can manage is 0 medals. Note also, the sixth point estimate is less than zero, this cannot happen as a nation cannot get minus medals.

## (c) Calculating MSE and MAPE

Provided you have a vector containing the predicted values (say `pred`) and a vector containing the observed values (say `obs`), the MSE can be calculated using the following: 

`mean((pred-obs)^2)`. 

Similarly, the MAPE can be calculated using the following: 

`100*mean(abs((pred-obs)/obs))`. 

So, in this example where do we get `pred` and `obs` from? 

One solution, and the one we will use in this notebook, results from noting that we can obtain just the predictions, and not the prediction intervals, by omitting the argument `type = "interval"` from the `predict()` command. Also, the vector `medals` in the data frame `new2016DataFrame` gives the number of medals that each nation actually did win at Rio 2016 and hence are the observed values. 

### (i)

Run the following code to obtain the vectors `pred` and `obs`and display the first six observations of `pred`. Check that `pred` appears to be correct by comparing them with what you obtained in part (b)(iii).

In [ ]:
pred <- predict(medalsMLRFit, newdata = olympic2016)
obs <- olympic2016[,'medals']
head(pred)

#### Solution <a class="tocSkip">

The values do indeed match the first column, `fit`, of the output obtained in part (b)(iii).

### (ii)

Using the empty code cell below, obtain the MSE and MAPE for this case (we will explain what the values you obtain mean next).

#### Solution <a class="tocSkip">

The required code and resulting output are shown below.

In [6]:
# MSE
mean((pred - obs)^2)
# MAPE
100*mean(abs((pred - obs)/obs))

[1] 9.949917

[1] Inf

###  <a class="tocSkip">

You should have found that the MSE in this case is 9.95 (to two decimal places). Note that by itself this information is only of limited help, it becomes more helpful when comparing predictions from different models.

For MAPE the value is given as "Inf", which is what R uses to indicate when a value is infinitely big. The problem here is that many of the nations did not actually win any medals, and so, at times, when trying to calculate the MAPE we are dividing by zero. 

## (d) Considering the prediction intervals

Calculating the percentage of observations that end up within the 95% prediction interval is a bit more complicated. 

We will do this in two steps: 

* First we will get R to calculate a vector in which every entry is either `TRUE` or `FALSE`. The $i$th entry in this vector will be `TRUE` if the observed number of medals for the $i$th nation is in its corresponding prediction interval, and `FALSE` otherwise. 
* We will then use the `table()` command using this vector of `TRUE` and `FALSE` values as its argument. The `table()` command produces a basic frequency table for the values in the vector and so we can see how many `TRUE` values there are (i.e. how many of the actual observations lie in their prediction intervals) and how many `FALSE` values there are (i.e. how many of the actual observations do *not* lie in their prediction intervals).   

### (i)

Run the code below to calculate how many times the actual observation lies in the corresponding 95% prediction interval. 

(Note here we use `predictions2016` which includes the predictions *and* the 95% prediction intervals instead of `pred`.)

In [ ]:
# Work out whether each entry in obs is in corresponding prediction interval
inInterval <- (obs > predictions2016[, 'lwr']) & (obs < predictions2016[,'upr'])
# Tabulate inInterval
table(inInterval)

### (ii)

From the output you obtained in part (i), do the prediction intervals seem reasonable?

#### Solution <a class="tocSkip">

From the table, we can see that 181 of the (181+11) observations lie within their corresponding 95% prediction intervals. In other words, 94.3% lie within the corresponding 95% prediction intervals. This means that the prediction intervals seem reasonable. If anything, they might be a little bit narrow because not quite enough observations are lie within their correspoind predictions.